In [916]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [917]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.spatial.transform import Rotation
import cv2
import pandas as pd
import plotly.graph_objects as go


In [918]:
K = [ 909.95300569, 0., 635.79822139, 0., 909.95300569, 385.66617804, 0., 0., 1. ]
K = np.array(K).reshape(3, 3)

In [919]:
MOCK_DATA = False
MOCK_DATA_NOISY = True

In [920]:
num_screws = 5

if MOCK_DATA:
    # Generate mock data
    screw_center = np.array([1, 0, 0])
    screw_circle_radius = 0.05
    screw_positions = []
    for i in range(num_screws):
        ang = i * 2*np.pi / num_screws
        screw_positions.append(screw_center + np.array([screw_circle_radius*np.cos(ang), screw_circle_radius*np.sin(ang), 0]))
    mock_screw_positions = np.stack(screw_positions)

    # Mock poses
    num_pictures = 15
    cam_base_pos = [1, 0, 0.5]
    cam_rot = Rotation.from_euler("xyz",[180, 0, 0],degrees=True).as_matrix()
    poses = np.stack([np.eye(4) for i in range(num_pictures)])
    for i in range(num_pictures):
        rand_ang = np.random.random()*2*np.pi
        rand_rotm = np.array([[np.cos(rand_ang), np.sin(rand_ang)],[-np.sin(rand_ang), np.cos(rand_ang)]])
        cam_pos_deviation = np.dot(rand_rotm,np.array([0,1]))
        cam_pos_deviation = 0.1*np.array([*cam_pos_deviation, 0])
        poses[i][:3,3] = cam_base_pos
        poses[i][:3,3] += cam_pos_deviation
        poses[i][:3,:3] = cam_rot
    # Pose given as cam2world

    # Mock detections
    detections = np.zeros((num_pictures, num_screws, 2))
    for i in range(num_pictures):
        cam_pose = poses[i]
        screw_positions_img = np.zeros((num_screws,2))
        for j in range(num_screws):
            screw_position_world_hom = np.hstack([mock_screw_positions[j],1])
            screw_position_cam_hom = np.dot(np.linalg.inv(cam_pose), screw_position_world_hom)
            screw_position_cam = screw_position_cam_hom[:3] / screw_position_cam_hom[-1]
            screw_position_img_hom = np.dot(K, screw_position_cam)
            screw_positions_img[j,:] = screw_position_img_hom[:2] / screw_position_img_hom[-1]
            
            if MOCK_DATA_NOISY:
                # Add some noise
                screw_positions_img[j,:] += np.random.normal(scale=5, size=2)
        img = np.zeros((720, 1280, 3), dtype=np.uint8)
        for (u, v) in screw_positions_img:
            u_i, v_i = int(round(u)), int(round(v))
            if 0 <= u_i < img.shape[1] and 0 <= v_i < img.shape[0]:
                cv2.circle(img, (u_i, v_i), radius=6, color=(0, 0, 255), thickness=2)
        plt.figure(figsize=(5, 5))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title("Mock screw positions (image)")
        plt.show()

        detections[i, :] = screw_positions_img

    poses = np.repeat(poses,num_screws,axis=0)
    poses = [poses[i] for i in range(poses.shape[0])]
    img = np.repeat(list(range(num_pictures)),num_screws,axis=0)
    x = detections[:,:,0].flatten()
    y = detections[:,:,1].flatten()
    data = {
        "img": img,
        "pose": poses,
        "x": x,
        "y": y
    }

In [921]:
if not MOCK_DATA:
    # Open collected data
    with open("data/20260213_171657/_screw_detections.csv", "r") as f:
        data = f.read().splitlines()
        headers = data[0].split(",")
        data = data[1:]  # Remove header
        data_dict = {header: [] for header in headers}
        for row in data:
            values = row.split(",")
            for header, value in zip(headers, values):
                data_dict[header].append(value)
    # Make pose into TF matrices
    poses = data_dict["pose"]
    for i in range(len(poses)):
        pose = poses[i]
        pose = [float(x.split(">")[1]) for x in pose.split(";")]
        pos =  pose[:3]
        quat = pose[3:]
        matrix = np.eye(4)
        matrix[:3, 3] = pos
        matrix[:3, :3] = Rotation.from_quat(quat).as_matrix()
        data_dict["pose"][i] = matrix
    data = data_dict
    timestamps = np.unique(data["timestamp"])
    timestamp_ids = {timestamp: i for i, timestamp in enumerate(timestamps)}
    data["img"] = [timestamp_ids[t] for t in data["timestamp"]]
    num_pictures = len(timestamps)

In [922]:
data["ray_origin"] = []
data["ray_direction"] = []
data["ray_num"] = []

ray_idx = 0
last_img_idx = 0
for img_idx, u, v, pose in zip(data["img"], data["x"], data["y"], data["pose"]):
    u = float(u)
    v = float(v)
    pixel_homogeneous = np.array([u, v, 1.0])
    ray_direction = np.linalg.inv(K) @ pixel_homogeneous
    ray_direction /= np.linalg.norm(ray_direction)  # Normalize the direction
    camera2world_pose = pose
    world2camera_pose = np.linalg.inv(camera2world_pose)
    camear2world_rot = Rotation.from_matrix(camera2world_pose[:3,:3])

    ray_direction_rot = camear2world_rot.apply(ray_direction)
    ray_origin = camera2world_pose[:3, 3]  # Extract the translation part of the pose
    data["ray_origin"].append(ray_origin)
    data["ray_direction"].append(ray_direction_rot)

    if img_idx == last_img_idx:
        data["ray_num"].append(ray_idx)
        ray_idx += 1
    else:
        ray_idx = 0
        data["ray_num"].append(ray_idx)
        ray_idx += 1
        
    last_img_idx = img_idx

In [923]:
data = pd.DataFrame.from_dict(data)
data

,timestamp,class,x,y,width,height,confidence,depth,pose,img,ray_origin,ray_direction,ray_num
0,20260213_171730,0,656.539,352.796,23,24,0.84,0,"[[0.1746210627339813, 0.1435490538394718, 0.97...",0,"[0.6176, 0.2336, 0.5795]","[0.9720240655697592, -0.13359847541784747, -0....",0
1,20260213_171730,0,519.594,535.706,24,26,0.83,410,"[[0.1746210627339813, 0.1435490538394718, 0.97...",0,"[0.6176, 0.2336, 0.5795]","[0.9549383049474554, -0.29670133311232505, -0....",1
2,20260213_171730,0,474.069,327.928,23,22,0.81,0,"[[0.1746210627339813, 0.1435490538394718, 0.97...",0,"[0.6176, 0.2336, 0.5795]","[0.9177707100167679, -0.3218460795533206, -0.2...",2
3,20260213_171730,0,360.377,544.787,26,27,0.78,402,"[[0.1746210627339813, 0.1435490538394718, 0.97...",0,"[0.6176, 0.2336, 0.5795]","[0.8933562850836595, -0.4491928086519796, -0.0...",3
4,20260213_171730,0,323.189,401.37,25,26,0.74,413,"[[0.1746210627339813, 0.1435490538394718, 0.97...",0,"[0.6176, 0.2336, 0.5795]","[0.8667583701714205, -0.47180105009683626, -0....",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,20260213_172335,0,652.891,267.919,30,30,0.86,0,"[[0.31609028477958034, 0.22035476377397262, 0....",14,"[0.6252, 0.1889, 0.4543]","[0.8926098706491113, -0.29607121371906375, -0....",0
68,20260213_172335,0,764.992,528.839,30,31,0.82,310,"[[0.31609028477958034, 0.22035476377397262, 0....",14,"[0.6252, 0.1889, 0.4543]","[0.9805545745848321, -0.18412442224084255, -0....",1
69,20260213_172335,0,902.59,260.228,30,31,0.81,324,"[[0.31609028477958034, 0.22035476377397262, 0....",14,"[0.6252, 0.1889, 0.4543]","[0.9371276602732199, -0.03655553028718803, -0....",2
70,20260213_172335,0,487.137,394.399,32,36,0.67,0,"[[0.31609028477958034, 0.22035476377397262, 0....",14,"[0.6252, 0.1889, 0.4543]","[0.8617942160953446, -0.4680156385281409, -0.1...",3


In [924]:
def plot_poses(poses, scale=0.05, axis_labels=True, pose_labels=None, ax=None, figsize=(8, 6)):
    """
    Visualize 4x4 pose matrices as coordinate frames using Plotly.
    - poses: single 4x4 array or iterable/array of shape (N,4,4)
    - scale: length of drawn axes
    - axis_labels: annotate axes ('x','y','z') for each pose
    - ax: ignored (kept for compatibility)
    Returns the plotly Figure.
    """
    poses = np.asarray(poses)
    if poses.ndim == 2 and poses.shape == (4, 4):
        poses = poses[None, ...]
    assert poses.ndim == 3 and poses.shape[1:] == (4, 4), "poses must be (N,4,4) or (4,4)"

    fig = go.Figure()
    all_points = []

    pose_labels = pose_labels if pose_labels is not None else list(range(len(poses)))

    for i, pose in zip(pose_labels, poses):
        t = pose[:3, 3]
        R = pose[:3, :3]
        for vec, color, lbl in zip(R.T, ("red", "green", "blue"), ("x", "y", "z")):
            end = t + vec * scale
            line = np.vstack([t, end])
            fig.add_trace(go.Scatter3d(
                x=line[:, 0], y=line[:, 1], z=line[:, 2],
                mode="lines", line=dict(color=color, width=4), showlegend=False
            ))
            fig.add_trace(go.Scatter3d(
                x=[t[0]], y=[t[1]], z=[t[2]],
                mode="markers", marker=dict(color=color, size=4), showlegend=False
            ))
            if axis_labels:
                fig.add_trace(go.Scatter3d(
                    x=[end[0]], y=[end[1]], z=[end[2]],
                    mode="text", text=[f"{lbl}{i}"], textposition="top center",
                    showlegend=False
                ))
        all_points.extend([t, t + R[:, 0] * scale, t + R[:, 1] * scale, t + R[:, 2] * scale])

    all_points = np.vstack(all_points)
    mins = all_points.min(axis=0)
    maxs = all_points.max(axis=0)
    centers = 0.5 * (mins + maxs)
    radius = (maxs - mins).max() / 2.0

    fig.update_layout(
        scene=dict(
            xaxis=dict(title="X", range=[centers[0] - radius, centers[0] + radius]),
            yaxis=dict(title="Y", range=[centers[1] - radius, centers[1] + radius]),
            zaxis=dict(title="Z", range=[centers[2] - radius, centers[2] + radius]),
        ),
        width=figsize[0] * 100,
        height=figsize[1] * 100,
    )
    # fig.show()
    return fig

# Only keep one pose per image
unique_img_df = data.groupby("img").first()
poses = unique_img_df["pose"].values
img_idxes = unique_img_df.index.values

plot_poses(np.stack(poses), pose_labels=img_idxes)

In [925]:
import numpy as np
from scipy.spatial.distance import cdist

# 1. Extract unique camera positions
img_indices = data["img"].unique()
camera_positions = []
for img in img_indices:
    pos = data[data["img"] == img]["pose"].iloc[0][:3, 3]
    camera_positions.append(pos)
camera_positions = np.array(camera_positions)

# --- CONFIGURATION ---
N_TO_KEEP = num_pictures // 3  # Keep one-third of the total pictures
N_TOTAL = len(img_indices)

if N_TOTAL <= N_TO_KEEP:
    selected_indices = list(range(N_TOTAL))
    print(f"Keeping all {N_TOTAL} images as they are fewer than N_TO_KEEP.")
else:
    # --- Farthest Point Sampling (FPS) ---
    
    # Start with the two cameras that are furthest apart globally
    dist_matrix = cdist(camera_positions, camera_positions, metric='euclidean')
    i, j = np.unravel_index(np.argmax(dist_matrix), dist_matrix.shape)
    
    selected_indices = [i, j]
    remaining_indices = list(set(range(N_TOTAL)) - set(selected_indices))
    
    while len(selected_indices) < N_TO_KEEP:
        # Get coordinates of points we have and points we don't
        pts_selected = camera_positions[selected_indices]
        pts_remaining = camera_positions[remaining_indices]
        
        # Calculate distance from every 'remaining' point to every 'selected' point
        # Result shape: (len(remaining), len(selected))
        dists_to_selected = cdist(pts_remaining, pts_selected, metric='euclidean')
        
        # For each remaining point, find the distance to its NEAREST selected neighbor
        # (We want to pick the point that is 'most alone' or furthest from the current set)
        min_dists = np.min(dists_to_selected, axis=1)
        
        # The next point to add is the one that has the largest 'minimum distance'
        best_idx_in_remaining = np.argmax(min_dists)
        best_global_idx = remaining_indices[best_idx_in_remaining]
        
        selected_indices.append(best_global_idx)
        remaining_indices.pop(best_idx_in_remaining)

# 2. Filter the Dataframe
keep_img_ids = img_indices[selected_indices]
to_remove = [img for img in img_indices if img not in keep_img_ids]

data = data[data["img"].isin(keep_img_ids)].copy()
img_indices = data["img"].unique()

print(f"Successfully selected {len(keep_img_ids)} images with optimized spatial baseline.")
print("Removed due to redundancy:", to_remove)

Successfully selected 5 images with optimized spatial baseline.
Removed due to redundancy: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(7), np.int64(8), np.int64(10), np.int64(11), np.int64(12), np.int64(13)]


In [926]:
# Plot filtered poses
unique_img_df = data.groupby("img").first()
poses = unique_img_df["pose"].values
img_idxes = unique_img_df.index.values

plot_poses(np.stack(poses), pose_labels=img_idxes)

In [927]:
def closest_point_between_rays(
    ray1_origin, ray1_direction, ray2_origin, ray2_direction, point_in_front=False, point_on_ray1=False
):
    # Compute the cross product of the direction vectors
    cross = np.cross(ray1_direction, ray2_direction)
    denom = np.linalg.norm(cross) ** 2

    # If the rays are parallel (cross product is zero), return None
    if denom < 1e-6:
        return None, None

    # Compute the parameters for the closest points on each ray
    t1 = np.linalg.det([ray2_origin - ray1_origin, ray2_direction, cross]) / denom
    t2 = np.linalg.det([ray2_origin - ray1_origin, ray1_direction, cross]) / denom

    # If point_in_front is True, ensure t1 and t2 are >= 0 (i.e., only consider points in the positive direction)
    if point_in_front:
        t1 = max(t1, 0)
        t2 = max(t2, 0)

    # Compute the closest points on each ray
    closest_point_ray1 = ray1_origin + t1 * ray1_direction
    closest_point_ray2 = ray2_origin + t2 * ray2_direction

    # Get the distance between the closest points
    distance = np.linalg.norm(closest_point_ray1 - closest_point_ray2)

    # Return the midpoint between the two closest points as the best estimate
    closest_point = (closest_point_ray1 + closest_point_ray2) / 2
    if point_on_ray1:
        closest_point = closest_point_ray1
    return closest_point, distance

def distance_of_point_from_ray(point_3D, ray_origin, ray_direction):
    ray_to_point = point_3D - ray_origin
    ray_vector = np.dot(ray_to_point, ray_direction) * ray_direction
    perpendicular_vector = ray_to_point - ray_vector

    distance = np.linalg.norm(perpendicular_vector)
    return distance

In [928]:
def reprojection_error(point3D_hyp, point2D_meas, pose):
    point3D_hyp_hom = np.hstack((point3D_hyp, 1))
    point3D_hyp_hom = np.dot(pose, point3D_hyp_hom)
    point3D_hyp = point3D_hyp[:3] / point3D_hyp[-1]
    point2D_hyp = np.dot(K, point3D_hyp)
    point2D_hyp = point2D_hyp[:2] / point2D_hyp[-1]
    error = np.linalg.norm(point2D_hyp, point2D_meas)
    return error

Ray pairing between maximum distance images

In [929]:

# Visualize the rays and the closest point

def plot_rays(ray_origins, ray_directions, colors="red", name="ray"):
    if isinstance(colors, str):
        colors = [colors for i in range(ray_origins.shape[0])]
        print(colors)
    fig = go.Figure()
    for o, d, color in zip(ray_origins, ray_directions, colors):
        line = np.vstack([o, o + d])
        fig.add_trace(go.Scatter3d(
            x=line[:, 0], y=line[:, 1], z=line[:, 2],
            mode="lines", line=dict(color=color), name=name, showlegend=False
        ))
        fig.add_trace(go.Scatter3d(
            x=[o[0]], y=[o[1]], z=[o[2]],
            mode="markers", marker=dict(color=color, size=4), showlegend=False
        ))
    fig.update_layout(scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"))
    fig.show()

def plot_rays_and_points(ray_origins, ray_directions, points=None, colors="red", name="ray", point_color="green", point_size=6):
    if isinstance(colors, str):
        colors = [colors for _ in range(ray_origins.shape[0])]
    fig = go.Figure()
    for o, d, color in zip(ray_origins, ray_directions, colors):
        line = np.vstack([o, o + d])
        fig.add_trace(go.Scatter3d(
            x=line[:, 0], y=line[:, 1], z=line[:, 2],
            mode="lines", line=dict(color=color), name=name, showlegend=False
        ))
        fig.add_trace(go.Scatter3d(
            x=[o[0]], y=[o[1]], z=[o[2]],
            mode="markers", marker=dict(color=color, size=4), showlegend=False
        ))
    if points is not None:
        points = np.asarray(points)
        fig.add_trace(go.Scatter3d(
            x=points[:, 0], y=points[:, 1], z=points[:, 2],
            mode="markers", marker=dict(color=point_color, size=point_size), name="points", showlegend=False
        ))
    fig.update_layout(scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"))
    fig.show()

def plot_two_rays_with_closest(ray_o1, ray_d1, ray_o2, ray_d2, closest_point):
    ray1_line = np.vstack([ray_o1, ray_o1 + ray_d1])
    ray2_line = np.vstack([ray_o2, ray_o2 + ray_d2])

    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=ray1_line[:, 0], y=ray1_line[:, 1], z=ray1_line[:, 2],
                               mode="lines", line=dict(color="red"), name="ray1"))
    fig.add_trace(go.Scatter3d(x=ray2_line[:, 0], y=ray2_line[:, 1], z=ray2_line[:, 2],
                               mode="lines", line=dict(color="blue"), name="ray2"))
    fig.add_trace(go.Scatter3d(x=[ray_o1[0]], y=[ray_o1[1]], z=[ray_o1[2]],
                               mode="markers", marker=dict(color="red", size=5), name="ray1_origin"))
    fig.add_trace(go.Scatter3d(x=[ray_o2[0]], y=[ray_o2[1]], z=[ray_o2[2]],
                               mode="markers", marker=dict(color="blue", size=5), name="ray2_origin"))
    fig.add_trace(go.Scatter3d(x=[closest_point[0]], y=[closest_point[1]], z=[closest_point[2]],
                               mode="markers", marker=dict(color="green", size=6), name="closest_point"))
    fig.update_layout(scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"))
    fig.show()


In [930]:
img_data1 = data[data["img"] == np.random.choice(img_indices)]
img_data2 = data[data["img"] == np.random.choice(img_indices)]

closest_points = {}
for idx1, detection1 in img_data1.iterrows():
    ray_o1 = detection1["ray_origin"]
    ray_d1 = detection1["ray_direction"]

    for idx2, detection2 in img_data2[["ray_origin", "ray_direction"]].iterrows():
        ray_o2 = detection2["ray_origin"]
        ray_d2 = detection2["ray_direction"]

        closest_point, distance = closest_point_between_rays(
            np.array(ray_o1),
            np.array(ray_d1),
            np.array(ray_o2),
            np.array(ray_d2),
        )
        if distance is None:
            closest_points[(idx1, idx2)] = (ray_o1, 0)
        else:
            closest_points[(idx1, idx2)] = (closest_point, distance)

print(closest_points)
# plot_two_rays_with_closest(ray_o1, ray_d1, ray_o2, ray_d2, closest_point)

{(31, 0): (array([0.39875328, 0.25437161, 0.57312918]), np.float64(0.09909405465783667)), (31, 1): (array([-0.05449079,  0.43536991,  0.58938771]), np.float64(0.0160859726462658)), (31, 2): (array([0.92724141, 0.07581617, 0.47066837]), np.float64(0.1061581707396132)), (31, 3): (array([0.39353662, 0.29431293, 0.55499228]), np.float64(0.10759216998991697)), (31, 4): (array([ 1.32968293, -0.14699028,  0.41355649]), np.float64(0.06761160075131006)), (31, 5): (array([0.49244757, 0.19400954, 0.53498952]), np.float64(0.05490549659315555)), (32, 0): (array([0.54161898, 0.22219885, 0.5430463 ]), np.float64(0.10898084358015026)), (32, 1): (array([0.34986296, 0.3153622 , 0.53780376]), np.float64(0.08781034244714664)), (32, 2): (array([0.63570193, 0.18444292, 0.53168749]), np.float64(0.11194569372998493)), (32, 3): (array([0.01479952, 0.52974396, 0.54063098]), np.float64(0.09446671283978218)), (32, 4): (array([0.87467052, 0.04121458, 0.51054396]), np.float64(0.09799563400362013)), (32, 5): (array(

In [931]:
cmap = plt.get_cmap()
colors = cmap(data["img"]/max(data["img"]))
all_ray_origins = np.concatenate([data["ray_origin"].to_list()])
all_ray_directions = np.concatenate([data["ray_direction"].to_list()])

plot_rays(
    all_ray_origins, 
    all_ray_directions,
    colors=colors
)


Ray pairing between random images

In [932]:
# screw_cands = []
# matched_rays = []
# assert len(img_data1["img"].unique()) == 1, "img_data1 should contain only one unique image"
# other_image_indices = data[data["img"]!=img_data1["img"].unique()[0]]["img"].unique()
# base_img_idx = img_data1["img"].unique()[0]

# for _, row in img_data1.iterrows():
#     # Get original ray, we iterate through all of them for this image
#     base_ray_num = row["ray_num"]
#     ray_o = row["ray_origin"]
#     ray_d = row["ray_direction"]

#     # Iterate through other images
#     min_dist_cands = []
#     min_dists = []
#     min_dist_ray_indices = []
#     for idx in other_image_indices:
#         img_data = data[data["img"]==idx]

#         dists = []
#         min_dist_cand = None
#         min_dist = float("inf")
#         min_dist_ray_idx = None

#         # Iterate through rays of the other image to get closest intersection
#         for sub_idx, sub_row in img_data.iterrows():
#             sub_ray_o = sub_row["ray_origin"]
#             sub_ray_d = sub_row["ray_direction"]
#             cand, dist = closest_point_between_rays(
#                 np.array(ray_o),
#                 np.array(ray_d),
#                 np.array(sub_ray_o),
#                 np.array(sub_ray_d),
#                 point_in_front=True,
#                 point_on_ray1=True,
#             )
#             dists.append(dist)

#             # Keep the candidate with the smallest distance to the ray
#             if dist < min_dist:
#                 min_dist = dist
#                 min_dist_cand = cand
#                 min_dist_ray_idx = sub_idx
#         min_dist_cands.append(min_dist_cand)
#         min_dists.append(min_dist)
#         min_dist_ray_indices.append(min_dist_ray_idx)
    
#     dist_sums = []
#     assert len(min_dist_cands) == len(other_image_indices) == len(min_dist_ray_indices)

#     # For all candidates on the original ray, compute distance from all rays of other images and sum up
#     #   there are as many candidates as other images
#     for cand, ray_idx in zip(min_dist_cands, min_dist_ray_indices):
#         dist_sum = 0
#         for idx in other_image_indices:
#             img_data = data[data["img"]==idx]
#             for sub_idx, sub_row in img_data.iterrows():
#                 dist = distance_of_point_from_ray(
#                     cand,
#                     np.array(sub_row["ray_origin"]),
#                     np.array(sub_row["ray_direction"]),
#                 )
#                 dist_sum += dist
#         dist_sums.append(dist_sum)
#     best_cand_idx = np.argmin(dist_sums)
#     best_cand = min_dist_cands[best_cand_idx]

#     # Lastly, find the rays of the other images that are the closest (matching rays)
#     matched_indices = []
#     for idx in other_image_indices:
#         img_data = data[data["img"]==idx]

#         min_dist_ray_idx = None
#         min_dist = float("inf")
#         for sub_idx, sub_row in img_data.iterrows():
#             ray_idx = sub_row["ray_num"]
#             dist = distance_of_point_from_ray(
#                 best_cand,
#                 np.array(sub_row["ray_origin"]),
#                 np.array(sub_row["ray_direction"]),
#             )
#             if dist < min_dist:
#                 min_dist = dist
#                 min_dist_ray_idx = ray_idx
#         matched_indices.append(min_dist_ray_idx)
        
#     matched_rays.append((base_ray_num, matched_indices))
#     screw_cands.append(best_cand)

# print(screw_cands)
# print(matched_rays)
# print(other_image_indices)

In [933]:
# screw_points = []
# # Run least square optimization to minimize reprojection error of the candidates
# for screw_cand, ray_groups in zip(screw_cands, matched_rays):
#     base_ray_num, matched_indices = ray_groups
#     # Get the corresponding rays for the matched indices
#     base_ray_o = img_data1[img_data1["ray_num"]==base_ray_num]["ray_origin"].values[0]
#     base_ray_d = img_data1[img_data1["ray_num"]==base_ray_num]["ray_direction"].values[0]

#     matched_rays_data = [(base_ray_o, base_ray_d)]
#     for i, ray_idx in enumerate(matched_indices):
#         other_img_idx = other_image_indices[i]
#         other_img_data = data[data["img"]==other_img_idx]
#         ray_o = other_img_data[other_img_data["ray_num"]==ray_idx]["ray_origin"].values[0]
#         ray_d = other_img_data[other_img_data["ray_num"]==ray_idx]["ray_direction"].values[0]
#         matched_rays_data.append((ray_o, ray_d))
    
#     # Now we have the rays, we can run least square optimization to minimize reprojection error of the candidates
#     def residual_func(point3D):
#         error_sum = 0
#         for ray_o, ray_d in matched_rays_data:
#             error_sum += distance_of_point_from_ray(
#                 point3D,
#                 ray_o,
#                 ray_d,
#             )
#         return error_sum
#     result = scipy.optimize.minimize(residual_func, screw_cand)
#     print(result)
#     screw_points.append(result.x)
# print(screw_points)

In [934]:
from scipy.cluster.hierarchy import linkage, fcluster
import pandas as pd

# --- USER PARAMETER ---
N_SCREWS = num_screws  # Set the known number of screws here
# ----------------------

MAX_RAY_DIST = 0.05  # Max distance between rays
CLUSTER_DIST = 0.05  # Spatial grouping distance

candidate_points = []
candidate_ray_indices = [] 
candidate_ray_distances = [] # Store ray-to-ray distance for scoring

unique_imgs = data["img"].unique()

for i in range(len(unique_imgs)):
    for j in range(i + 1, len(unique_imgs)):
        img_a, img_b = unique_imgs[i], unique_imgs[j]
        rays_a = data[data["img"] == img_a]
        rays_b = data[data["img"] == img_b]
        
        for _, row_a in rays_a.iterrows():
            for _, row_b in rays_b.iterrows():
                pt, dist = closest_point_between_rays(
                    row_a["ray_origin"], row_a["ray_direction"],
                    row_b["ray_origin"], row_b["ray_direction"],
                    point_in_front=True
                )
                
                if pt is not None and dist < MAX_RAY_DIST:
                    # Optional: Depth filter (Z > 0)
                    # if pt[2] > 0:
                    candidate_points.append(pt)
                    candidate_ray_distances.append(dist)
                    candidate_ray_indices.append([
                        (row_a["img"], row_a["ray_num"]),
                        (row_b["img"], row_b["ray_num"])
                    ])

candidate_points = np.array(candidate_points)

if len(candidate_points) > 0:
    Z = linkage(candidate_points, method='single') 
    labels = fcluster(Z, t=CLUSTER_DIST, criterion='distance')
    print(f"Clustering found {len(np.unique(labels))} potential locations.")
else:
    labels = np.array([])
    print("No valid intersections found.")

cluster_stats = []

for label in np.unique(labels):
    indices = np.where(labels == label)[0]
    
    # Calculate Metrics
    contributing_rays = []
    for idx in indices:
        contributing_rays.extend(candidate_ray_indices[idx])
    
    unique_rays = set(contributing_rays)
    unique_images = set([r[0] for r in unique_rays])
    avg_ray_dist = np.mean([candidate_ray_distances[idx] for idx in indices])
    
    cluster_stats.append({
        'label': label,
        'num_images': len(unique_images),
        'num_intersections': len(indices),
        'precision': avg_ray_dist,
        'indices': indices,
        'unique_rays': unique_rays
    })

# Sort by: 1. Max Cameras, 2. Max Intersections, 3. Min Ray Distance (Precision)
cluster_stats.sort(key=lambda x: (-x['num_images'], -x['num_intersections'], x['precision']))

# Select only the top N
best_clusters = cluster_stats[:N_SCREWS]
print(f"Selected the {len(best_clusters)} best clusters based on camera agreement.")

Clustering found 19 potential locations.
Selected the 5 best clusters based on camera agreement.


In [935]:
from scipy.optimize import least_squares

final_screw_positions = []

for cluster in best_clusters:
    # 1. Gather all rays involved in this cluster
    target_rays_o = []
    target_rays_d = []
    for img_idx, ray_num in cluster['unique_rays']:
        row = data[(data["img"] == img_idx) & (data["ray_num"] == ray_num)].iloc[0]
        target_rays_o.append(row["ray_origin"])
        target_rays_d.append(row["ray_direction"])
        
    target_rays_o = np.array(target_rays_o)
    target_rays_d = np.array(target_rays_d)

    # 2. Initial Guess: Centroid of intersections
    initial_guess = np.mean(candidate_points[cluster['indices']], axis=0)

    # 3. Objective function: Distances from point to all rays
    def ray_residuals(p):
        res = []
        for o, d in zip(target_rays_o, target_rays_d):
            vec = p - o
            dist = np.linalg.norm(vec - np.dot(vec, d) * d)
            res.append(dist)
        return np.array(res)

    # 4. Refine using Huber Loss (ignore outliers beyond 1cm)
    res = least_squares(ray_residuals, initial_guess, loss='huber', f_scale=0.01)
    final_screw_positions.append(res.x)

final_screw_positions = np.array(final_screw_positions)

# Visual Verification
plot_rays_and_points(
    ray_origins=all_ray_origins, 
    ray_directions=all_ray_directions, 
    points=final_screw_positions,
    point_color="blue"
)